<ul class="breadcrumb">
  <li><a href="1_Conventional_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Conventional)</a></li>
  <li><a href="2_Univariate_Linear_Additive_Genetic_Model.ipynb">Bayesian Linear Additive Genetic Model</a></li> 
  <li><a href="3_Genomic_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Genomic Data)</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Bayesian Linear Mixed Models (Genomic Data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 1: Load Packages</button> 

In [2]:
using JWAS,JWAS.Datasets,DataFrames,CSV

<button type="button" class="btn btn-lg btn-primary">Step 2: Read data</button> 

In [3]:
phenofile  = Datasets.dataset("example","phenotypes.txt")
pedfile    = Datasets.dataset("example","pedigree.txt")
genofile   = Datasets.dataset("example","genotypes.txt")

phenotypes = CSV.read(phenofile,delim = ',',header=true)
pedigree   = get_pedigree(pedfile,separator=",",header=true);

coding pedigree...  10%|███                             |  ETA: 0:00:01

Finished!


coding pedigree... 100%|████████████████████████████████| Time: 0:00:00


In [4]:
head(phenotypes)

,ID,y1,y2,y3,x1,x2,x3,dam
1,a1,-0.06,3.58,-1.18,0.9,2,m,0
2,a2,-0.6,4.9,0.88,0.3,1,f,0
3,a3,-2.07,3.19,0.73,0.7,2,f,0
4,a4,-2.63,6.97,-0.83,0.6,1,m,a2
5,a5,2.31,3.5,-1.52,0.4,2,m,a2
6,a6,0.93,4.87,-0.01,5.0,2,f,a3


<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Univariate Linear Mixed Model (Genomic data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 3: Build Model Equations</button> 

In [5]:
model_equation1  ="y1 = intercept + x1*x3 + x2 + x3 + ID + dam";

In [6]:
R      = 1.0
model1 = build_model(model_equation1,R);

<button type="button" class="btn btn-lg btn-primary">Step 4: Set Factors or Covariates</button> 

In [7]:
set_covariate(model1,"x1");

<button type="button" class="btn btn-lg btn-primary">Step 5: Build Random or Fixed Effects</button> 

In [8]:
G1 = 1.0
G2 = eye(2)
set_random(model1,"x2",G1);
set_random(model1,"ID dam",pedigree,G2);

<button type="button" class="btn btn-lg btn-primary">Step 6: Use Genomic Information</button> 

In [9]:
G3 =1.0
add_genotypes(model1,genofile,G3,separator=',');

5 markers on 10 individuals were added.


<button type="button" class="btn btn-lg btn-primary">Step 7: Run Analysis</button> 

In [10]:
out1=runMCMC(model1,phenotypes,methods="BayesC",chain_length=5000,output_samples_frequency=100);

The prior for marker effects variance was calculated from genetic varaince and π.
Marker effects variance is 
0.462963



MCMC Information:

methods                                      BayesC
chain_length                                   5000
burnin                                            0
starting_value                                false
printout_frequency                             5001
output_samples_frequency                        100
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Information for hyper-parameter: π (Π)
π                                               0.0
estimatePi                                    false

Degree of freedom for hyper-parameters:
residual variances:                           4.000
iid random effect variances:                  4.000
polygenic effect variances:                   4.000
marker effect variances:                      4.00

running MCMC for BayesC0...100%|████████████████████████| Time: 0:00:01


<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multivariate Linear Mixed Model (Genomic data)</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 3: Build Model Equations</button> 

In [11]:
model_equation2 ="y1 = intercept + x1 + x3 + ID + dam
                  y2 = intercept + x1 + x2 + x3 + ID
                  y3 = intercept + x1 + x1*x3 + x2 + ID";

In [12]:
R      = eye(3)
model2 = build_model(model_equation2,R);

<button type="button" class="btn btn-lg btn-primary">Step 4: Set Factors or Covariates</button> 

In [13]:
set_covariate(model2,"x1");

<button type="button" class="btn btn-lg btn-primary">Step 5: Build Random or Fixed Effects</button> 

In [14]:
G1 = eye(2)
G2 = eye(4)
set_random(model2,"x2",G1);
set_random(model2,"ID dam",pedigree,G2);

INFO: x2 is not found in model equation 1.
INFO: dam is not found in model equation 2.
INFO: dam is not found in model equation 3.


<button type="button" class="btn btn-lg btn-primary">Step 6: Use Genomic Information</button> 

In [15]:
G3 = eye(3)
add_genotypes(model2,genofile,G3,separator=',');

5 markers on 10 individuals were added.


<button type="button" class="btn btn-lg btn-primary">Step 7: Run Analysis</button> 

In [16]:
out2=runMCMC(model2,phenotypes,methods="BayesC",estimatePi=true,chain_length=5000,output_samples_frequency=100);

 The prior for marker effects covariance matrix were calculated from genetic covariance matrix and π.
Marker effects covariance matrix is 

0.462963  0.0       0.0     
 0.0       0.462963  0.0     
 0.0       0.0       0.462963


MCMC Information:

methods                                      BayesC
chain_length                                   5000
burnin                                            0
starting_value                                false
printout_frequency                             5001
output_samples_frequency                        100
constraint                                    false
missing_phenotypes                            false
update_priors_frequency                           0

Information for hyper-parameter: π (Π)
π                              Dict([1.0, 0.0, 1.0]=>0.0,[0.0, 0.0, 1.0]=>0.0,[0.0, 1.0, 1.0]=>0.0,[1.0, 1.0, 0.0]=>0.0,[0.0, 0.0, 0.0]=>0.0,[0.0, 1.0, 0.0]=>0.0,[1.0, 0.0, 0.0]=>0.0,[1.0, 1.0, 1.0]=>1.0)
estimatePi                           

running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:05


<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [17]:
using JWAS:misc

In [19]:
get_breeding_values(model2,"MCMC_samples_for_marker_effects_y1.txt",
    "MCMC_samples_for_marker_effects_y2.txt",
    "MCMC_samples_for_marker_effects_y3.txt")

3-element Array{Any,1}:
 10×3 DataFrames.DataFrame
│ Row │ ID  │ EBV       │ PEV      │
├─────┼─────┼───────────┼──────────┤
│ 1   │ a1  │ 0.14087   │ 0.215545 │
│ 2   │ a2  │ 0.601516  │ 0.218892 │
│ 3   │ a3  │ -0.560584 │ 0.447752 │
│ 4   │ a4  │ -1.47249  │ 0.468224 │
│ 5   │ a5  │ 2.21488   │ 0.859895 │
│ 6   │ a6  │ -1.54531  │ 0.534377 │
│ 7   │ a7  │ 1.1256    │ 0.513053 │
│ 8   │ a8  │ -1.80605  │ 0.58949  │
│ 9   │ a9  │ -1.21175  │ 0.426753 │
│ 10  │ a10 │ 2.51333   │ 1.08296  │            
 10×3 DataFrames.DataFrame
│ Row │ ID  │ EBV        │ PEV      │
├─────┼─────┼────────────┼──────────┤
│ 1   │ a1  │ -0.201976  │ 0.25006  │
│ 2   │ a2  │ 0.167264   │ 0.505339 │
│ 3   │ a3  │ -0.0312766 │ 0.487881 │
│ 4   │ a4  │ 1.13785    │ 0.463578 │
│ 5   │ a5  │ -1.17256   │ 0.72546  │
│ 6   │ a6  │ 1.20283    │ 0.479842 │
│ 7   │ a7  │ -1.43608   │ 0.753506 │
│ 8   │ a8  │ 1.26539    │ 0.460513 │
│ 9   │ a9  │ 1.07529    │ 0.558253 │
│ 10  │ a10 │ -2.00672   │ 1.28574  │
 10×3 Data